In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [2]:
# для google colab
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# розмір картинки
IMG_SIZE = (75, 75)

# папка з датасетом у якій підпапки з фотографіями кожного символа 
# назви класів будуть відповідати іменам підпапок
directory = "images/"

In [4]:
data_generator = ImageDataGenerator(validation_split = 0.2, rescale = 1.0 / 255)
 
# датасет для тренування
train = data_generator.flow_from_directory(directory,
                                               target_size = IMG_SIZE,
                                               class_mode = 'categorical',
                                               subset = 'training')

# датасет для тестування
test = data_generator.flow_from_directory(directory,
                                               target_size = IMG_SIZE,
                                               class_mode = 'categorical',
                                               subset = 'validation')

Found 286 images belonging to 29 classes.
Found 55 images belonging to 29 classes.


In [5]:
def createModel(image_shape = IMG_SIZE):
    '''
    Аргументи:
        image_shape -- ширина та висота картинки
    Повертає:
        tf.keras.model
    '''
    input_shape = image_shape + (3,)
    
    base_model = InceptionV3(input_shape = input_shape,
                            include_top = False,
                            weights = "imagenet",
                            classes = 29,
                            classifier_activation = "softmax")
    # заморозимо шари, щоб їх не тренувати
    base_model.trainable = False

    # вхідний шар
    inputs = tf.keras.Input(shape = input_shape) 

    # для нормалізації інпута [-1;1]
    x = tf.keras.applications.inception_v3.preprocess_input(inputs) 

    x = base_model(x, training = False) 
    x = tfl.GlobalAveragePooling2D()(x) 
    x = tfl.Dropout(0.2)(x)
    outputs = tfl.Dense(29, activation = "softmax")(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [6]:
model = createModel(IMG_SIZE)

Metal device set to: Apple M1


2022-11-06 18:19:20.325972: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-06 18:19:20.326078: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# метрики
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision_f1 = precision(y_true, y_pred)
    recall_f1 = recall(y_true, y_pred)
    return 2 * ((precision_f1 * recall_f1) / (precision_f1 + recall_f1 + K.epsilon()))

In [8]:
base_learning_rate = 0.01

initial_epochs = 10

loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
optimizer = tf.keras.optimizers.Adam(base_learning_rate)
metrics = ['acc', f1_score, precision, recall]

In [9]:
model.compile(optimizer = optimizer,
              loss = loss_function,
              metrics = metrics)

In [10]:
history = model.fit(train,
                    validation_data = test,
                    epochs = initial_epochs)

Epoch 1/10


2022-11-06 18:19:22.090928: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
/Users/tylerdurden/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/backend.py:5531: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2022-11-06 18:19:23.480631: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - ETA: 0s - loss: 3.3898 - acc: 0.3951 - f1_score: 0.4210 - precision: 0.5697 - recall: 0.3403         

2022-11-06 18:19:25.008714: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 4s 217ms/step - loss: 3.3898 - acc: 0.3951 - f1_score: 0.4210 - precision: 0.5697 - recall: 0.3403 - val_loss: 0.8330 - val_acc: 0.8545 - val_f1_score: 0.8739 - val_precision: 0.9095 - val_recall: 0.8410
Epoch 2/10
9/9 [==============================] - 1s 63ms/step - loss: 1.0251 - acc: 0.7657 - f1_score: 0.7891 - precision: 0.8672 - recall: 0.7243 - val_loss: 0.3569 - val_acc: 0.9091 - val_f1_score: 0.9077 - val_precision: 0.9655 - val_recall: 0.8567
Epoch 3/10
9/9 [==============================] - 1s 56ms/step - loss: 0.4813 - acc: 0.8741 - f1_score: 0.8830 - precision: 0.9285 - recall: 0.8426 - val_loss: 0.1577 - val_acc: 0.9636 - val_f1_score: 0.9565 - val_precision: 0.9565 - val_recall: 0.9565
Epoch 4/10
9/9 [==============================] - 1s 62ms/step - loss: 0.2620 - acc: 0.9231 - f1_score: 0.9281 - precision: 0.9524 - recall: 0.9056 - val_loss: 0.2179 - val_acc: 0.9273 - val_f1_score: 0.9253 - val_precision: 0.9253 - val_recall: 0.925

In [11]:
# model.summary()

# print("Кількість шарів у базовій моделі: ", len(tunedModel.layers))

In [12]:
tunedModel = model.layers[3]
tunedModel.trainable = True

fine_tune_start = 250

In [13]:
# Заморозимо всі шари перед fine_tune_start
for layer in tunedModel.layers[:fine_tune_start]:
    layer.trainable = False

optimizer = Adam(learning_rate = 0.0001 * base_learning_rate)

fine_tune_epochs = 100
total_epochs =  initial_epochs + fine_tune_epochs

In [14]:
model.compile(optimizer = optimizer,
              loss = loss_function,
              metrics = metrics)

In [15]:
history_fine = model.fit(train,
                         validation_data = test,
                         epochs = total_epochs,
                         initial_epoch = history.epoch[-1])

Epoch 10/110


2022-11-06 18:19:32.161962: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - ETA: 0s - loss: 0.0393 - acc: 0.9930 - f1_score: 0.9945 - precision: 0.9963 - recall: 0.9928

2022-11-06 18:19:34.640217: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 5s 277ms/step - loss: 0.0393 - acc: 0.9930 - f1_score: 0.9945 - precision: 0.9963 - recall: 0.9928 - val_loss: 0.1272 - val_acc: 0.9818 - val_f1_score: 0.9889 - val_precision: 1.0000 - val_recall: 0.9783
Epoch 11/110
9/9 [==============================] - 1s 95ms/step - loss: 0.0411 - acc: 0.9930 - f1_score: 0.9931 - precision: 0.9931 - recall: 0.9931 - val_loss: 0.1271 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 12/110
9/9 [==============================] - 1s 83ms/step - loss: 0.0391 - acc: 0.9965 - f1_score: 0.9947 - precision: 0.9964 - recall: 0.9931 - val_loss: 0.1281 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 13/110
9/9 [==============================] - 1s 89ms/step - loss: 0.0270 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1285 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall:

9/9 [==============================] - 1s 88ms/step - loss: 0.0247 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1439 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 43/110
9/9 [==============================] - 1s 83ms/step - loss: 0.0191 - acc: 0.9965 - f1_score: 0.9963 - precision: 0.9963 - recall: 0.9963 - val_loss: 0.1429 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 44/110
9/9 [==============================] - 1s 85ms/step - loss: 0.0161 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1438 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 45/110
9/9 [==============================] - 1s 83ms/step - loss: 0.0264 - acc: 0.9930 - f1_score: 0.9931 - precision: 0.9931 - recall: 0.9931 - val_loss: 0.1453 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 

9/9 [==============================] - 1s 94ms/step - loss: 0.0194 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1517 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 75/110
9/9 [==============================] - 1s 97ms/step - loss: 0.0242 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1531 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 76/110
9/9 [==============================] - 1s 88ms/step - loss: 0.0183 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1542 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 77/110
9/9 [==============================] - 1s 84ms/step - loss: 0.0255 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1523 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 

9/9 [==============================] - 1s 84ms/step - loss: 0.0229 - acc: 0.9965 - f1_score: 0.9963 - precision: 0.9963 - recall: 0.9963 - val_loss: 0.1585 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 107/110
9/9 [==============================] - 1s 82ms/step - loss: 0.0148 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1612 - val_acc: 0.9818 - val_f1_score: 0.9844 - val_precision: 0.9844 - val_recall: 0.9844
Epoch 108/110
9/9 [==============================] - 1s 85ms/step - loss: 0.0209 - acc: 0.9965 - f1_score: 0.9963 - precision: 0.9963 - recall: 0.9963 - val_loss: 0.1632 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recall: 0.9783
Epoch 109/110
9/9 [==============================] - 1s 88ms/step - loss: 0.0173 - acc: 0.9965 - f1_score: 0.9965 - precision: 0.9965 - recall: 0.9965 - val_loss: 0.1618 - val_acc: 0.9818 - val_f1_score: 0.9783 - val_precision: 0.9783 - val_recal

In [16]:
model.save('Fine_tuned_model01.h5', overwrite = True, include_optimizer = True)